In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
import os
import glob

from PIL import Image
from lxml import etree

# New Section

In [ ]:
def boundingBox(f):
  tree = etree.parse(f)
  for dim in tree.xpath("size"):
    width = int(dim.xpath("width")[0].text)
    height = int(dim.xpath("height")[0].text)
  for dim in tree.xpath("object/bndbox"):
    xmin = int(dim.xpath("xmin")[0].text)
    ymin = int(dim.xpath("ymin")[0].text)
    xmax = int(dim.xpath("xmax")[0].text)
    ymax = int(dim.xpath("ymax")[0].text)
  return [int(xmin), int(ymin),int(xmax), int(ymax)]

In [ ]:
img_dir = "./db/images"
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
files.sort()

path = './db/annotations/'
text_files = ['./db/annotations/'+f for f in sorted(os.listdir(path))]

inputs = []
n = len(files)

for i in range(n):
  image_file = files[i]
  box_file = text_files[i]

  im = Image.open(image_file)
  bbox = boundingBox(box_file)

  im = im.crop(bbox)
  im = im.resize((150, 75))

  inputs.append(np.array(im))

In [ ]:
for i in range(3):
  plt.imshow(inputs[i])
  plt.show()

In [ ]:
bw_inputs = []
for i in range(len(inputs)):
  bw_inputs.append(cv2.cvtColor(inputs[i], cv2.COLOR_BGR2GRAY))
bw_inputs = np.array(bw_inputs)

In [ ]:
from skimage import data, transform, color
from sklearn.feature_extraction.image import PatchExtractor

# imgs_to_use = ['camera', 'text', 'coins', 'moon',
#                'page', 'clock',
#                'grass', 'gravel', 'brick']
# images = [color.rgb2gray(getattr(data, name)())
#           for name in imgs_to_use]
images = []

img1 = cv2.imread('./db/nolicenseplate.jpeg')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('./db/nolicenseplate2.jpeg')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
images.append(img1)
images.append(img2)

In [ ]:
#i've basically added some more negative patches from the car images itself(stuff outside the bounding boxes) to avoid false positives.. seems to work
for i in range(25):
  image_file = files[i]
  box_file = text_files[i]

  im = Image.open(image_file)
  bbox = boundingBox(box_file)

  im1 = cv2.cvtColor(np.array(im.crop([0, 0, bbox[0], bbox[1]])), cv2.COLOR_BGR2GRAY)
  im2 = cv2.cvtColor(np.array(im.crop([bbox[2], bbox[3], im.size[0], im.size[1]])), cv2.COLOR_BGR2GRAY)

  if(im1.shape[0] > 75 and im1.shape[1] > 150):
    images.append(im1)
  if(im2.shape[0] > 75 and im2.shape[1] > 150):
    images.append(im2)


In [ ]:
def extract_patches(img, N, scale=1.0, patch_size=(75,150)):
    # print(img.shape, scale)
    # print(scale*np.array(patch_size))
    # extracted_patch_size = tuple((scale * np.array(patch_size)).astype(int))
    extractor = PatchExtractor(patch_size=patch_size,
                               max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch, patch_size)
                            for patch in patches])
    return patches

negative_patches = np.vstack([extract_patches(im, 500, scale)
                              for im in images for scale in [1.0, 2.0]])

In [ ]:
negative_patches.shape

In [ ]:
bw_inputs.shape

In [ ]:
from skimage.feature import hog  
from itertools import chain

X = np.array([im for im in chain(bw_inputs, negative_patches)])

In [ ]:
y = np.zeros(X.shape[0])
y[:433] = 1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=21)

In [ ]:
X_train_hog = [hog(x) for x in X_train]
X_test_hog = [hog(x) for x in X_test]

X_train_hog = np.array(X_train_hog)
X_test_hog = np.array(X_test_hog)

In [ ]:
X_train_hog.shape

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV 

grid = GridSearchCV(LinearSVC(dual=False), {'C': [1.0, 2.0, 4.0, 8.0]},cv=3)
grid.fit(X_train_hog, y_train)
grid.best_score_

In [ ]:
model = grid.best_estimator_
model.fit(X_train_hog, y_train)

In [ ]:
for i in range(len(X_test)):
  fds = X_test_hog[i].reshape(1, -1)
  pred = model.predict(fds)
  if(pred == 1):
    plt.imshow(X_test[i])
    plt.show()

In [ ]:
camera = color.rgb2gray(getattr(data, 'camera')())
plt.imshow(camera)
plt.show()

In [ ]:
# Intersection Over Union [ratio of I/U]
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

def plot_scores_IOU(iou):
    no_of_items = range(43);
    thresh = [0.5 for i in range(43)];
    plt.plot(no_of_items, iou, 'g', label='IOU')
    plt.plot(no_of_items, thresh, 'b', label="threshold")
    plt.title('IOU Scores')
    plt.legend()
    plt.show()
    


In [ ]:
#got this from the internet
def non_max_suppression_slow(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# initialize the list of picked indexes
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
  # keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list, add the index
		# value to the list of picked indexes, then initialize
		# the suppression list (i.e. indexes that will be deleted)
		# using the last index
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]
  # loop over all indexes in the indexes list
		for pos in range(0, last):
			# grab the current index
			j = idxs[pos]
			# find the largest (x, y) coordinates for the start of
			# the bounding box and the smallest (x, y) coordinates
			# for the end of the bounding box
			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])
			# compute the width and height of the bounding box
			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)
			# compute the ratio of overlap between the computed
			# bounding box and the bounding box in the area list
			overlap = float(w * h) / area[j]
			# if there is sufficient overlap, suppress the
			# current bounding box
			if overlap > overlapThresh:
				suppress.append(pos)
		# delete all indexes from the index list that are in the
		# suppression list
		idxs = np.delete(idxs, suppress)
	# return only the bounding boxes that were picked
	return boxes[pick]

In [ ]:
iou2 = []
for k in range(400):
  test_img = Image.open(files[k])
  box_file = text_files[k]
  bbox = boundingBox(box_file)
  pick = []

  for scale in range(2,15):
    boxes = []
    w, h = test_img.size
    col_inc = h//(scale*2)
    row_inc = w//scale

    image = np.array(test_img)
    for i in range(0, w, max(10, row_inc)):
      for j in range(0, h, max(10, col_inc)):
        if i+row_inc <= w and j+col_inc <= h:
          test = test_img.crop((i, j, i+row_inc, j+col_inc))
          test = test.resize((150, 75))

          window = cv2.cvtColor(np.array(test), cv2.COLOR_BGR2GRAY)
          window = hog(window)
          window = window.reshape(1,-1)
          pred = model.predict(window)

          if pred == 1:
            box = [i, j, i+row_inc, j+col_inc]
            boxes.append(box)

    boxes = np.array(boxes)
    nms = non_max_suppression_slow(boxes, 0.3)
    for box in nms:
      pick.append(box)
    if(len(pick) > 0):
      break

  max_iou = 0
  for box in pick:
    iou = bb_intersection_over_union(box, bbox)
    if iou > max_iou:
      best = box
      max_iou = iou

    x1, y1, x2, y2 = box
    image = cv2.rectangle(image, (x1, y1, x2-x1, y2-y1), (255, 0, 0))
  print(k + 1, max_iou)
  iou2.append(max_iou)
    

  plt.imshow(image)
  plt.show()

In [ ]:
print(bb_intersection_over_union((220, 120, 435, 170), bbox))

In [ ]:
def plot_scores_IOU(iou):
    no_of_items = range(400);
    thresh = [0.5 for i in range(400)];
    plt.plot(no_of_items, iou, 'g', label='IOU')
    plt.plot(no_of_items, thresh, 'b', label="threshold")
    plt.title('IOU Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores_IOU(iou2)